# The Battle of Neighborhoods

## Introduction

This project is to take a glimpse at different neighborhoods of Richmond,VA. Richmond is the are capital city of Virginia and is major city in middle Eastern part of the US.
This might be an interest for travelers and Business owners from the different part of the world who want to visit Richmond for various reasons.

## Data

I organized data set of zipcodes and the city neighborhoods with latitude & longtude for the city from multiple online sources.

## Methodology

Learnings from the previous modules and the sources majorly relied on Foursquare API to retrieve all venues of each neighborhoods, then group by each neighborhood and to count how many venues before filter top 10 most common venue types of each neighborhoods.

In [17]:
import numpy as np
import pandas as pd

In [18]:
df=pd.read_excel("Richmond.xlsx")
df.head()

,Zipcode,Neighborhood,Latitude,longitude
0,23218,Richmond City,38.71,-85.37
1,23219,Richmond City,38.71,-85.37
2,23220,Richmond City,37.47,-82.75
3,23221,Richmond City,36.76,-83.58
4,23222,Richmond City,37.86,-83.37


In [19]:
df['Zipcode'] = df['Zipcode'].astype(str)

In [20]:
df.shape

(39, 4)

In [21]:
from geopy.geocoders import Nominatim
import folium

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
# Working with Foursquare
CLIENT_ID = '15N1TQXP1GA4YDPO234G2DRQKFPW4FQKIWCJPP2AH5ZCBGBW' # your Foursquare ID
CLIENT_SECRET = '1F5AHUFKEADX45CQ0OQZUAKWMQST3LIRHGV3Z2H3M0AGWU3R' # your Foursquare Secret
VERSION = '20190604' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 15N1TQXP1GA4YDPO234G2DRQKFPW4FQKIWCJPP2AH5ZCBGBW
CLIENT_SECRET:1F5AHUFKEADX45CQ0OQZUAKWMQST3LIRHGV3Z2H3M0AGWU3R


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
venues=getNearbyVenues(df['Neighborhood'], df['Latitude'], df['longitude'])

In [26]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Richmond City,38.71,-85.37,Weight Watchers - Milton School Apartments,38.706930,-85.373603,Weight Loss Center
1,Richmond City,38.71,-85.37,Weight Watchers - Milton School Apartments,38.706930,-85.373603,Weight Loss Center
2,Richmond City,37.47,-82.75,Minnie Park,37.471912,-82.752800,Park
3,Richmond City,37.47,-82.75,Hi-Lite Pizza,37.472296,-82.753864,Pizza Place
4,Richmond City,36.84,-84.84,Rite Aid,36.840980,-84.839144,Pharmacy


In [27]:
venues.shape

(92, 7)

In [28]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Chesterfield County,12,12,12,12,12,12
Henrico County,6,6,6,6,6,6
Richmond City,74,74,74,74,74,74


In [29]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,ATM,American Restaurant,Department Store,Discount Store,Entertainment Service,Fast Food Restaurant,Food,Gas Station,Gift Shop,Grocery Store,Gym,Hotel,Insurance Office,Liquor Store,Park,Pharmacy,Pizza Place,Sandwich Place,Steakhouse,Weight Loss Center
0,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [30]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,ATM,American Restaurant,Department Store,Discount Store,Entertainment Service,Fast Food Restaurant,Food,Gas Station,Gift Shop,Grocery Store,Gym,Hotel,Insurance Office,Liquor Store,Park,Pharmacy,Pizza Place,Sandwich Place,Steakhouse,Weight Loss Center
0,Chesterfield County,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000
1,Henrico County,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000
2,Richmond City,0.081081,0.054054,0.013514,0.067568,0.081081,0.054054,0.081081,0.013514,0.013514,0.054054,0.081081,0.013514,0.027027,0.013514,0.013514,0.162162,0.040541,0.027027,0.081081,0.027027


In [31]:
# Define a function for top venus for a neighborhood
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gift Shop,American Restaurant,Department Store,Discount Store,Fast Food Restaurant,Food
1,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gift Shop,American Restaurant,Department Store,Discount Store,Fast Food Restaurant,Food
2,Richmond City,Pharmacy,ATM,Gym,Entertainment Service,Food,Steakhouse,Discount Store,American Restaurant,Fast Food Restaurant,Grocery Store


In [35]:
# set number of clusters
kclusters = 3

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 1], dtype=int32)